In [2]:
import pandas as pd
df = pd.read_csv("../data/diabetes_raw.csv")


In [3]:
print(df.head())
print(df.info())
print(df.describe())
print(df.shape)
print("Data loaded successfully.")

   Diabetes_binary  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0              0.0     1.0       1.0        1.0  40.0     1.0     0.0   
1              0.0     0.0       0.0        0.0  25.0     1.0     0.0   
2              0.0     1.0       1.0        1.0  28.0     0.0     0.0   
3              0.0     1.0       0.0        1.0  27.0     0.0     0.0   
4              0.0     1.0       1.0        1.0  24.0     0.0     0.0   

   HeartDiseaseorAttack  PhysActivity  Fruits  ...  AnyHealthcare  \
0                   0.0           0.0     0.0  ...            1.0   
1                   0.0           1.0     0.0  ...            0.0   
2                   0.0           0.0     1.0  ...            1.0   
3                   0.0           1.0     1.0  ...            1.0   
4                   0.0           1.0     1.0  ...            1.0   

   NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex   Age  Education  \
0          0.0      5.0      18.0      15.0       1.0  0.0   9.0   

In [4]:
df.columns


Index(['Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

Select only the needed features

In [ ]:
selected_cols = [
    "BMI",
    "HighBP",
    "Age",
    "Smoker",
    "PhysActivity",
    "Diabetes_binary"
]
#creates a new DataFrame containing ONLY the above columns.
df_selected = df[selected_cols]
df_selected.head()


,BMI,HighBP,Age,Smoker,PhysActivity,Diabetes_binary
0,40.0,1.0,9.0,1.0,0.0,0.0
1,25.0,0.0,7.0,1.0,1.0,0.0
2,28.0,1.0,9.0,0.0,0.0,0.0
3,27.0,1.0,11.0,0.0,1.0,0.0
4,24.0,1.0,11.0,0.0,1.0,0.0


Check missing values

In [6]:
df_selected.isna().sum()


BMI                0
HighBP             0
Age                0
Smoker             0
PhysActivity       0
Diabetes_binary    0
dtype: int64

Fix Invalid BMI Values (Zero BMI)

In [8]:
df_selected = df_selected.copy()
#Find the median(middle) BMI value that will be used to fill in missing values.
median_bmi = df_selected["BMI"].median()
# Replace all BMI=0 with median BMI
df_selected["BMI"] = df_selected["BMI"].replace(0, median_bmi)


Ensure All Binary Variables Are 0 or 1

In [ ]:
#It loops through each binary column and prints the unique values inside.
for col in ["HighBP", "Smoker", "PhysActivity", "Diabetes_binary"]:
    print(col, df_selected[col].unique())


HighBP [1. 0.]
Smoker [1. 0.]
PhysActivity [0. 1.]
Diabetes_binary [0. 1.]


converts the smallest value to 0
and the largest value to 1
and everything else in between.

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_selected[["BMI", "Age"]] = scaler.fit_transform(df_selected[["BMI", "Age"]])


Save the Clean Dataset

In [ ]:
df_selected.to_csv("../data/diabetes_clean.csv", index=False)
print("Cleaned dataset saved!")


Cleaned dataset saved!
